# Infrastructure

## Imports

In [1]:
import numpy as np
import pandas as pd
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import erlang  

## Loading Data

In [0]:
# data = pd.read_excel('/content/Analytic Model - 2nd Circle Policy.xlsx', use_cols=['C', 'E', 'G'])
# df_tsym = np.array(data['fsym'].dropna())
# df_tres = np.array(data['ftres'].dropna())
# dp_inf = np.array(data['pinf'].dropna())
n0 = 30

In [0]:
# fig = go.Figure()
# for d in [p_inf, dp_inf]:
#   fig.add_trace(go.Scatter(y=d))
# fig

## Probability Manipulations

In [0]:
def get_cdf(p):
  assert np.abs(p.sum()-1)<0.01, p
  return np.cumsum(p)

def get_pdf_of_sum(p1, p2):
  assert np.abs(p1.sum()-1)<0.01, p1
  assert np.abs(p2.sum()-1)<0.01, p2
  p = np.zeros(len(p1) + len(p2) - 1)
  for (i, p1i) in enumerate(p1):
    for (j, p1j) in enumerate(p2):
      p[i + j] += p1i * p1j
  p = p[:n0] 
  assert  np.abs(p.sum()-1)<0.01, p
  return p

def pad_distribution(p, n=n0):
  if (len(p) == n0):
    return p
  return np.pad(p, (0, n0 - len(p)), mode='constant', constant_values=(0, 0))

def get_semi_geometric_distribution(mean):
  means = [1.0, 2.0, 3.0, 4.0, 5.05, 6.15, 7.3, 8.7, 10.3] 
  p = 1 / means[mean]
  dist = np.array([p * (1 - p)**i for i in range(n0)])
  return dist/dist.sum()

def get_mean(p):
  return np.dot(np.arange(len(p)), p)

def get_samples(p, N=1):
  if (N == 1):
    return np.random.choice(a=len(p), p=p, size=1)[0]
  else:
    return np.random.choice(a=len(p), p=p, size=N)

def normalize_lst(lst):
  array = np.array(lst)
  return array/array.sum()

def get_erlang(mean, scale=2.0):
  if (mean == 0):
    return np.array([1] + [0] * (n0 - 1))
  cdf = erlang.cdf(np.linspace(0, n0 - 1, n0), a=mean/scale-1/scale**2, scale=scale)
  pdf = [cdf[0]] + [cdf[i] - cdf[i - 1] for i in range(1, len(cdf))]
  pdf[-1] += (1 - sum(pdf))
  return np.array(pdf)

def get_p_inf(f_tlat, f_tinf):
  N=10000
  t_lats = get_samples(f_tlat, N=N)
  t_infs = get_samples(f_tinf, N=N)
  beginnings = t_lats
  endings = t_lats + t_infs
  return np.array([((beginnings <= i) & (i <= endings)).sum()/N for i in range(0, max(max(endings), n0))])

In [5]:
f_tinf = get_erlang(2.9)
f_tlat = get_erlang(3.68)
p_inf = get_p_inf(f_tlat, f_tinf)

f_tsym = get_erlang(5.2) # incubation
f_tres = get_erlang(5)

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.2).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.59).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(2.35).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(2.25).



# Analytical Model Calculators

## Ai, Bij, Cijk Calculators

In [0]:
def get_calculators(scenario):
  mode = scenario['mode']
  f_tsym = scenario['f_tsym']
  f_tres = scenario['f_tres']
  p_sym = scenario['p_sym']
  p_star = scenario['p_star']
  p_0 = scenario['p_0']
  p_1 = scenario['p_1']
  p_2 = scenario['p_2']

  F_tsym = get_cdf(f_tsym)
  sum_cdf = get_cdf(get_pdf_of_sum(f_tsym, f_tres))

  def calc_branch1(days_since_1st_circle_infection,
                   days_since_2nd_circle_infection,
                   mode):
    if (mode == 0):
      return 1
    else:
      if (days_since_1st_circle_infection >= len(sum_cdf)):
        elem1 = 1 - p_1 * p_sym
      else:
        elem1 = 1 - p_1 * p_sym * sum_cdf[days_since_1st_circle_infection]
      if (mode == 1 or days_since_2nd_circle_infection == None):
        return elem1
      elif (mode == 2):
        if (days_since_2nd_circle_infection >= len(sum_cdf)):
          elem2 = 1 - p_2 * p_sym
        else:
          elem2 = 1 - p_2 * p_sym * sum_cdf[days_since_2nd_circle_infection]
        return elem1 * elem2
    
  def calc_branch2(days_since_infection):
    prob_symp = p_sym * F_tsym[days_since_infection]
    if (prob_symp == 0):
      elem1 = 0
    else:
      conditioned_sum_cdf = get_cdf(get_pdf_of_sum(f_tsym[:days_since_infection+1] / F_tsym[days_since_infection], f_tres))
      elem1 = prob_symp * (1 - p_0 * conditioned_sum_cdf[days_since_infection]) * (1 - p_star)
    elem2 = 1 - prob_symp
    return elem1 + elem2

  def calc_Ai(i):
    if (i >= n0): #patient is no more infectious
      return 0
    return p_inf[i] * calc_branch2(i) # i = number of days since infection of patient-0
  
  def calc_Bij(i, j):
    if ((j - i >= n0) or (i >= n0)): # patient is no longer infectious
      return 0
    if (j < i): # this case makes no sense
      return 0
    else:
      branch2 = calc_branch2(j - i) # j-i = number of days since infection of patient-1
      branch1 = calc_branch1(j, None, mode)
      return p_inf[j - i] * branch1 * branch2
  
  def calc_Cijk(i, j, k):
    if ((k - j >= n0) or (j - i >= n0) or (i >= n0)): # patient is no longer infectious
      return 0
    if ((j < i) or (k < j)): # this case makes no sense
      return 0
    branch2 = calc_branch2(k - j) # k-j = number of days since infection of patient-2
    branch1 = calc_branch1(k - i, k, mode) # k-i = num. of days since infection of patient-1, the first circle of patient-2. k = num. of days since infection of patient-0, the second 
    return p_inf[k - j] * branch1 * branch2
  return calc_Ai, calc_Bij, calc_Cijk

## Circles Calculator

In [0]:
def calc_circles(scenario,
                 filter_i=lambda i: 1, 
                 filter_j=lambda i, j: 1, 
                 filter_k=lambda i, j, k: 1):
  calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)
  n = np.nonzero(p_inf)[0].max() + 1
  sum1, sum2, sum3 = 0, 0, 0
  for i in range(n):
    if (filter_i(i) != 0):
      Ai = calc_Ai(i) * filter_i(i)
      sum1 += Ai 
      for j in range(i, 2 * n):
        if (filter_j(i, j) != 0):
          Bij = calc_Bij(i, j)  * filter_j(i, j)
          sum2 += Ai * Bij
          for k in range(j, 3 * n):
            if (filter_k(i, j, k) != 0):
              Cijk = calc_Cijk(i, j, k)   * filter_k(i, j, k)
              sum3 += Ai * Bij * Cijk
  t = scenario.copy()
  t['f_tsym'] = ''
  t['f_tres'] = ''
  t['name'] = ''
  t['s'] = ''
  return (1, sum1, sum2, sum3)

# Results Graphing

In [0]:
def build_scenario_df(scenarios, var_name, var_range):
  rows = []
  for s in scenarios:
    for var in var_range:
      row = {}
      for key in list(s.keys()):
        if (s[key] != 'VARIABLE'):
          row[key] = s[key]
        else:
          row[key] = var
      if (len(scenarios) == 1):
        if (var_name == 'f_tres'):
          m = get_mean(row['f_tres'])
          row['name'] = s['name'] + ' {}={}'.format(var_name, m)
      row['s'] = s
      cc = calc_circles(row)
      row['rel_R0'] = cc[3]/cc[2]
      rows.append(row)
  df = pd.DataFrame.from_dict(rows, orient='columns')


  base_scenario = df[df['name'] == df['name'].iloc[0]]
  if (var_name == 'p_sym'):
    normalizer = lambda row: base_scenario.rel_R0[base_scenario['p_sym'] == row['p_sym']].iloc[0]
  elif (var_name == 'f_tres'):
    normalizer = lambda row: base_scenario.rel_R0.iloc[0]

  for index, row in df.iterrows():
    cur_val = df.loc[index, 'rel_R0']
    df.loc[index, 'rel_R0'] = cur_val/normalizer(row)

  if (var_name == 'f_tres'):
    df['f_tres'] = df['f_tres'].apply(get_mean)
    df['f_tres'] = df['f_tres'].apply(round).astype(str)
    df['scenario_name'] = df['s'].apply(lambda s: s['name'])
    df['name'] = df['f_tres'].apply(lambda x: ' delay: {}'.format(x))

  return df

### Sensitivity Analysis of R0 regarding percentage of symptomatics

In [30]:
scenarios = [dict(name='0th Circle - without Symp. Iso.', mode=0, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=1.0, p_1=0.0, p_2=0.0),
             dict(name='0th Circle - with Symp. Iso.'   , mode=0, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=1.0, p_0=1.0, p_1=0.0, p_2=0.0),
             dict(name='1st Circle - without Symp. Iso.', mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=1.0, p_1=1.0, p_2=0.0),
             dict(name='1st Circle - with Symp. Iso.'   , mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=1.0, p_0=1.0, p_1=1.0, p_2=0.0),
             dict(name='2nd Circle - without Symp. Iso.', mode=2, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=1.0, p_1=1.0, p_2=1.0),
             dict(name='2nd Circle - with Symp. Iso.'   , mode=2, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=1.0, p_0=1.0, p_1=1.0, p_2=1.0)]
df = build_scenario_df(scenarios, 'p_sym', [0.65, 0.7, 0.75, 0.8, 0.85])
fig = px.bar(data_frame=df,
       x='p_sym',
       y='rel_R0',
       color='name',
       barmode='group',
       color_discrete_sequence=px.colors.sequential.Plasma_r,
       title='Relative R0 vs. Percentage of Symptomatics')
fig.update_layout(xaxis_title='Percentage of Symptomatics',
                  yaxis_title='Relative R0',
                  yaxis_tickformat = ',.0%',
                  xaxis_tickformat = ',.0%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



### Sensitivity Analysis of R0 regarding compliance

In [0]:
# p_sym = 0.75
# scenarios = []
# for compliance in ['low', 'high']:
#   for mode in [0, 1, 2]:
#     for symp_isolation in ['with', 'without']:
#       p_0 = 1.0
#       p_1 = 0.8
#       p_2 = 0.7
#       p_star = 0.75
#       if (compliance == 'low'):
#         for p in [p_0, p_1, p_2, p_star]:
#           p *= 0.8
#       scenarios.append(dict(name='{} Circle - {} Symp. Iso'.format(mode, symp_isolation),
#                             mode=mode,
#                             f_tsym=f_tsym,
#                             f_tres=f_tres,
#                             p_sym=p_sym,
#                             p_star=p_star,
#                             p_0 = p_0,
#                             p_1 = p_1,
#                             p_2 = p_2))
# for row in scenarios:
#   row['rel_R0'] = np.cbrt(calc_circles(row['mode'], row['f_tsym'], row['f_tres'], row['p_sym'], row['p_star'], row['p_0'], row['p_1'], row['p_2'])[3])
# df = pd.DataFrame.from_dict(scenarios, orient='columns')
# normalizer = df.rel_R0.iloc[0]
# df['rel_R0'] = df['rel_R0'] / normalizer
# fig = px.bar(data_frame=df,
#        x='p_0',
#        y='rel_R0',
#        color='name',
#        barmode='group',
#        color_discrete_sequence=px.colors.sequential.Plasma_r,
#        title='Relative R0 vs. Percentage of Symptomatics')
# fig.update_layout(xaxis_title='Avg. Delay in Results of Coronatests',
#                   yaxis_title='Relative R0',
#                   yaxis_tickformat = ',.0%')

In [27]:
scenarios = [dict(name='0th Circle - without Symp. Iso.', mode=0, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=0.8, p_1=0.0, p_2=0.0),
             dict(name='0th Circle - with Symp. Iso.'   , mode=0, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.2, p_0=0.8, p_1=0.0, p_2=0.0),
             dict(name='1st Circle - without Symp. Iso.', mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=0.8, p_1=0.6, p_2=0.0),
             dict(name='1st Circle - with Symp. Iso.'   , mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.2, p_0=0.8, p_1=0.6, p_2=0.0),
             dict(name='2nd Circle - without Symp. Iso.', mode=2, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.0, p_0=0.8, p_1=0.6, p_2=0.6),
             dict(name='2nd Circle - with Symp. Iso.'   , mode=2, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0.2, p_0=0.8, p_1=0.6, p_2=0.6)]
df = build_scenario_df(scenarios, 'p_sym', [0.65, 0.7, 0.75, 0.8, 0.85])
px.bar(data_frame=df,
       x='p_sym',
       y='rel_R0',
       color='name',
       barmode='group',
       color_discrete_sequence=px.colors.sequential.Plasma_r,
       title='Relative R0 vs. Percentage of Symptomatics')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



## Relative R0 vs. Delay in Results - 1st Circle Policy

In [28]:
scenarios = [dict(name='1st Circle Policy', mode=1, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.75, p_star=0.0, p_0=1.0, p_1=0.8, p_2=0.0)]
df = build_scenario_df(scenarios, 'f_tres', [get_erlang(i) for i in range(9)])
fig = px.bar(data_frame=df,
       x='f_tres',
       y='rel_R0',
       color='name',
       color_discrete_sequence=px.colors.sequential.RdBu_r,
       title='Relative R0 vs. Delay in Results -1st Circle Policy Without Isolation of Symptomatics')
fig.update_layout(xaxis_title='Avg. Delay in Results of Coronatests',
                  yaxis_title='Relative R0',
                  yaxis_tickformat = ',.0%')

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(2.25).

/usr/local/lib/python3.6/dist-packages/s

## Relative R0 vs. Delay in Results and Symptomatics Isolation

In [29]:
scenarios = [dict(name='1st Circle Policy', mode=1, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.75, p_star=0.00, p_0=1.0, p_1=0.8, p_2=0.0),
             dict(name='1st Circle Policy', mode=1, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.75, p_star=0.60, p_0=1.0, p_1=0.8, p_2=0.0)]
df = build_scenario_df(scenarios, 'f_tres', [get_erlang(i) for i in range(9)])
  

fig = px.bar(data_frame=df,
       x='p_star',
       y='rel_R0',
       color='name',
       barmode='group',
       color_discrete_sequence=px.colors.sequential.RdBu_r,
       title='Relative R0 vs. Symptomatics Isolation - 1st Circle Policy')
fig.update_layout(xaxis_title='Percentage of Symptomatics Isolated',
                  yaxis_title='Relative R0',
                  yaxis_tickformat = ',.0%',
                  xaxis_tickformat = ',.0%')

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(2.25).

/usr/local/lib/python3.6/dist-packages/s

## Relative Efficiency of Policies vs. Delay in Results

In [0]:
scenarios = [dict(name='1st Circle Policy', mode=1, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.7, p_star=0.0, p_0=1.0, p_1=0.8, p_2=0.0),
             dict(name='2nd Circle Policy', mode=2, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.7, p_star=0.0, p_0=1.0, p_1=0.8, p_2=0.8)]
var_range = [get_erlang(i) for i in range(9)]
df = build_scenario_df(scenarios, 'f_tres', var_range)
normalizer = pd.concat([df.rel_R0[:9], df.rel_R0[:9]], ignore_index=True)
df['rel_R0'] = df['rel_R0']/normalizer
fig = go.Figure()
fig.add_trace(go.Bar(x=df.f_tres[df['mode']==0], y=df.rel_R0[df['mode']==0], marker_color=px.colors.sequential.RdBu_r))
fig.add_trace(go.Bar(x=df.f_tres[df['mode']==1], y=df.rel_R0[df['mode']==1], marker_color=px.colors.sequential.RdBu_r))
fig.add_trace(go.Bar(x=df.f_tres[df['mode']==2], y=df.rel_R0[df['mode']==2], marker_color=px.colors.sequential.RdBu_r))
fig.update_layout(barmode='group',
                  title='Relative R0 vs. Delay in Results and Policy - Without Isolation of Symptomatics',
                  xaxis_title='Average Delay',
                  yaxis_title='Relative R0',
                  yaxis_tickformat = ',.0%')

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(0.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.25).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(1.75).

/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py:2847: RuntimeWarning:

The shape parameter of the erlang distribution has been given a non-integer value array(2.25).

/usr/local/lib/python3.6/dist-packages/s

In [0]:
scenarios = [dict(name='0th Circle Policy', mode=0, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.7, p_star=0.0, p_0=1.0, p_1=0.0, p_2=0.0),
             dict(name='1st Circle Policy', mode=1, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.7, p_star=0.8, p_0=1.0, p_1=0.8, p_2=0.0),
             dict(name='2nd Circle Policy', mode=2, f_tsym=f_tsym, f_tres='VARIABLE', p_sym=0.7, p_star=0.8, p_0=1.0, p_1=0.8, p_2=0.8)]
var_range = [get_erlang(i) for i in range(9)]
df = build_scenario_df(scenarios, 'f_tres', var_range)
px.bar(data_frame=df,
       x='mode',
       y='rel_R0',
       color='f_tres',
       barmode='group',
       color_discrete_sequence=px.colors.sequential.RdBu_r,
       title='Relative R0 vs. Delay in Results - Without Isolation of Symptomatics')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [0]:
scenarios = [dict(name='Base', mode=0, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0, p_0=1, p_1=1, p_2=1),
             dict(name='1st', mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym='VARIABLE', p_star=0, p_0=1, p_1=1, p_2=1)]
var_range = [0.2, 0.4, 0.6, 0.8, 1.0]
var_name = 'p_sym'
colors = px.colors.sequential.Plasma_r
title = 'Relative R0 vs. Percentage of Symptomatics - With Isolation of Symptomatics'
plot_comparison(scenarios, var_name, var_range, colors, title)

In [0]:
compliance_dict = {'low-compliance': 0.6, 'nominal-compliance': 0.8, 'high-compliance': 1.0}
delay_dict = {'delay: {}d'.format(i): i for i in range(10)}
policy_dict = {'1st Circle-Policy': (1, 0)}

# No Policy Scenarios
scenarios =[('1st Circle-Policy', 'delay: {}d'.format(i), 'nominal-compliance') for i in range(1, 10)]

p_sym = 0.7

rows = []
for s in scenarios:
  row = dict(name=' '.join(s[:2]),
            mode=policy_dict[s[0]][0],
            f_tsym=f_tsym,
            f_tres=get_erlang(delay_dict[s[1]]),
            p_star=policy_dict[s[0]][1],
            p_0=compliance_dict[s[2]],
            p_1=compliance_dict[s[2]],
            p_2=compliance_dict[s[2]],
            p_sym=p_sym,
            s=s)
  row['rel_R0'] = np.cbrt(calc_circles(row)[3])
  rows.append(row)

df = pd.DataFrame.from_dict(rows, orient='columns')
df['delay'] = df['s'].apply(lambda s: delay_dict[s[1]])

base_scenario = df[df['name'] == df['name'].iloc[0]]

for index, row in df.iterrows():
  cur_val = df.loc[index, 'rel_R0']
  normalizer = base_scenario.rel_R0.iloc[0]
  df.loc[index, 'rel_R0'] = cur_val/normalizer

px.bar(data_frame=df,
       x='delay',
       y='rel_R0',
       color='name',
       color_discrete_sequence=px.colors.sequential.RdBu_r,
       title='Relative R0 as a function of Delay; 70% Symptomatic, No Symptomatic Isolation')


In [0]:
compliance_dict = {'low-compliance': 0.6, 'nominal-compliance': 0.8, 'high-compliance': 1.0}
delay_dict = {'delay: {}d'.format(i): i for i in range(10)}
policy_dict = {'1st Circle-Policy': (1, 0.8)}

# No Policy Scenarios
scenarios =[('1st Circle-Policy', 'delay: {}d'.format(i), 'nominal-compliance') for i in range(1, 10)]

p_sym = 0.7

rows = []
for s in scenarios:
  row = dict(name=' '.join(s[:2]),
            mode=policy_dict[s[0]][0],
            f_tsym=f_tsym,
            f_tres=get_erlang(delay_dict[s[1]]),
            p_star=policy_dict[s[0]][1],
            p_0=compliance_dict[s[2]],
            p_1=compliance_dict[s[2]],
            p_2=compliance_dict[s[2]],
            p_sym=p_sym,
            s=s)
  row['rel_R0'] = np.cbrt(calc_circles(row)[3])
  rows.append(row)

df = pd.DataFrame.from_dict(rows, orient='columns')
df['delay'] = df['s'].apply(lambda s: delay_dict[s[1]])

base_scenario = df[df['name'] == df['name'].iloc[4]]

for index, row in df.iterrows():
  cur_val = df.loc[index, 'rel_R0']
  normalizer = base_scenario.rel_R0.iloc[0]
  df.loc[index, 'rel_R0'] = cur_val/normalizer

px.bar(data_frame=df,
       x='delay',
       y='rel_R0',
       color='name',
       color_discrete_sequence=px.colors.sequential.RdBu_r,
       title='Relative R0 as a function of Delay; 70% Symptomatic, With Symptomatic Isolation')


In [0]:
compliance_dict = {'low-compliance': 0.6, 'nominal-compliance': 0.8, 'high-compliance': 1.0}
delay_dict = {'low-delay': 0.66, 'nominal-delay': 1.0, 'high-delay': 1.5}
policy_dict = {'Base': (0, 0),
               '1st Circle-Policy': (1, 0),
               '2nd Circle-Policy': (2, 0)}

# No Policy Scenarios
scenarios =[('Base', 'nominal-delay', 'nominal-compliance'),
            ('1st Circle-Policy', 'nominal-delay', 'nominal-compliance'),
            ('2nd Circle-Policy', 'nominal-delay', 'nominal-compliance')]

f_tres_stats = np.random.choice(len(f_tres), p=f_tres, size=100000)
m, v = f_tres_stats.mean(), f_tres_stats.var()

rows = []
for s in scenarios:
  for p_sym in [0.2, 0.4, 0.6, 0.8, 1.0]:
    row = dict(name=s[0],
              mode=policy_dict[s[0]][0],
              f_tsym=f_tsym,
              f_tres=get_erlang(1),
              p_star=policy_dict[s[0]][1],
              p_0=compliance_dict[s[2]],
              p_1=compliance_dict[s[2]],
              p_2=compliance_dict[s[2]],
              p_sym=p_sym)
    row['rel_R0'] = np.cbrt(calc_circles(row)[3])
    rows.append(row)

df = pd.DataFrame.from_dict(rows, orient='columns')
df = df[['name', 'p_sym', 'rel_R0']]

base_scenario = df[df['name'] == df['name'].iloc[0]]

for index, row in df.iterrows():
  cur_val = df.loc[index, 'rel_R0']
  normalizer = base_scenario.rel_R0[base_scenario['p_sym'] == row['p_sym']].iloc[0]
  df.loc[index, 'rel_R0'] = cur_val/normalizer

px.bar(data_frame=df,
       x='p_sym',
       y='rel_R0',
       color='name',
       barmode='group',
       color_discrete_sequence=px.colors.sequential.Plasma_r,
       title='Relative R0 vs. Percentage of Symptomatics - No Isolation of Symptomatics')


## Cases Among 3rd Circle Until day-l

## Detection Day

In [0]:
def calculate_detection_statistics(scenario, N=3):
  patient_symptomatics_dist = np.array([1 - scenario['p_sym'], scenario['p_sym']])
  patient_detection_dist = scenario['f_tsym']
  calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)

  len_simulation = 1000
  sim_results = []
  for x in range(len_simulation):
    if (x % 100 == 0):
      print(x)
    c0_symptomatic = get_samples(patient_symptomatics_dist)
    if (c0_symptomatic):
      c0_detection_date = get_samples(patient_detection_dist)
      c0_zipped = [(c0_detection_date, 0)]
    else:
      c0_zipped = []

    c1_infection_dist = normalize_lst([calc_Ai(i) for i in range(num_days)])
    c1_infection_dates = get_samples(c1_infection_dist, N)
    c1_symptomatics = get_samples(patient_symptomatics_dist, N)
    c1_infection_of_symptomatics_dates = c1_infection_dates[c1_symptomatics == 1]
    if (len(c1_infection_of_symptomatics_dates) == 0):
      c1_zipped = []
    else:
      c1_detection_dates = c1_infection_of_symptomatics_dates + get_samples(patient_detection_dist, len(c1_infection_of_symptomatics_dates))
      c1_zipped = list(zip(c1_detection_dates, np.ones(len(c1_detection_dates))))

    c2_detection_dates = np.array([])
    for i in c1_infection_dates.astype(int):
      c2_infection_dist = normalize_lst([calc_Bij(i, j) for j in range(i, i + num_days)])
      c2_infection_dates = i + get_samples(c2_infection_dist, N)
      c2_symptomatics = get_samples(patient_symptomatics_dist, N)
      c2_infection_of_symptomatics_dates = c2_infection_dates[c2_symptomatics == 1]
      if (len(c2_infection_of_symptomatics_dates) == 0):
        c2_zipped = []
      else:
        c2_detection_dates = np.concatenate([c2_detection_dates, c2_infection_of_symptomatics_dates + get_samples(patient_detection_dist, len(c2_infection_of_symptomatics_dates))])
        c2_zipped = list(zip(c2_detection_dates, 2 * np.ones(len(c2_detection_dates))))

    detection_dates_with_circles = c0_zipped + c1_zipped + c2_zipped
    detection_dates_with_circles = sorted(detection_dates_with_circles, key=lambda x: x[0])
    detection_dates_with_circles = list(filter(lambda x: x[1] <= scenario['mode'], detection_dates_with_circles))
    if (len(detection_dates_with_circles) == 0):
      sim_results.append(None)
    else:
      sim_results.append(detection_dates_with_circles[0])
  res_samples = get_samples(f_tres, len_simulation)
  return_value = []
  for (i, elem) in enumerate(sim_results):
    if (elem == None):
      return_value.append((None, None))
    else:
      return_value.append((elem[0] + res_samples[i], elem[1]))
  return return_value

In [0]:
df = pd.DataFrame(columns=['0', '1', '2'])
for mode in [0, 1, 2]:
  scenario = dict(mode=mode, f_tsym=f_tsym, f_tres=f_tres, p_sym=0.75, p_star=0, p_0=1.0, p_1=0.8, p_2=0.0)
  stats = calculate_detection_statistics(scenario, 3)
  detection_statistics = pd.DataFrame(stats, columns=['day', 'circle'])
  chance_of_detection = lambda i: (detection_statistics.day <= i).sum()/len(detection_statistics)
  df[str(mode)] = [chance_of_detection(i) for i in range(30)]

0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900


In [0]:
fig = go.Figure([go.Scatter(x=df.index, y=df[str(i)]) for i in range(3)])
fig.add_trace(go.Scatter(x=df.index, y=pd.Series([calc_Ai(i) for i in range(len(p_inf))])))
fig.update_layout(title='Cumulative Probability of Quarantining Patient-0 vs. Time and Policy',
                  xaxis_title='Days',
                  yaxis_title='Cumulative Probability',
                  yaxis_tickformat='%')

In [0]:
for col in df.columns:
  df[col] = (1 - df[col]) * pd.Series(p_inf)

calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)
fig = go.Figure([go.Scatter(x=df.index, y=df[str(i)]) for i in range(3)])
fig.add_trace(go.Scatter(x=df.index, y=[calc_Ai(i) for i in df.index]))
fig.update_layout(title='Cumulative Probability of Quarantining Patient-0 vs. Time and Policy',
                  xaxis_title='Days',
                  yaxis_title='Cumulative Probability',
                  yaxis_tickformat='%')

In [0]:
def calc_circles(scenario,
                 filter_i=lambda i: 1, 
                 filter_j=lambda i, j: 1, 
                 filter_k=lambda i, j, k: 1):
  calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)
  n = np.nonzero(p_inf)[0].max() + 1
  sum1, sum2, sum3 = 0, 0, 0
  for i in range(n):
    if (filter_i(i) != 0):
      Ai = df[str(scenario['mode'])].iloc[i] * filter_i(i)
      sum1 += Ai 
      for j in range(i, 2 * n):
        if (filter_j(i, j) != 0):
          Bij = calc_Bij(i, j)  * filter_j(i, j)
          sum2 += Ai * Bij
          for k in range(j, 3 * n):
            if (filter_k(i, j, k) != 0):
              Cijk = calc_Cijk(i, j, k)   * filter_k(i, j, k)
              sum3 += Ai * Bij * Cijk
  return (1, sum1, sum2, sum3)

In [0]:
calc_circles(scenario)

(1, 2.973873626268481, 6.684591594969211, 15.776931783487363)

In [0]:
calc_circles(scenario)

(1, 3.2850769219877054, 7.596000763723803, 17.932992281947055)

In [0]:
scenario = dict(mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym=0.0, p_star=0, p_0=1.0, p_1=0.8, p_2=0.0)
stats = calculate_detection_statistics(scenario, 3)
detection_statistics = pd.DataFrame(stats, columns=['day', 'circle'])
circle0 = detection_statistics.day[detection_statistics.circle == 0]
circle1 = detection_statistics.day[detection_statistics.circle == 1]
circle2 = detection_statistics.day[detection_statistics.circle == 2]

0
100
200
300
400
500
600
700
800
900


In [0]:
fig = go.Figure()
def get_histogram(series):
  maximum = int(series.max())
  series_range = pd.Series(np.linspace(0, maximum + 1, maximum + 2))
  return series_range, series_range.apply(lambda i: len(series[series == i]))

fig = go.Figure()
for (i, series) in enumerate([circle0, circle1, circle2]):
  x, y = get_histogram(series)
  y = y/(len(np.concatenate([circle0, circle1, circle2])))
  fig.add_trace(go.Scatter(x=x, y=y, name='Circle {}'.format(i)))
fig
fig.update_layout(xaxis_title='Date of Discovery',
                  yaxis = dict(dtick=0.01),
                  yaxis_title='Percentage',
                  yaxis_tickformat = '%',
                  title='Distribution of the Circle & Date of Detection')

In [0]:
def calculate_detection_statistics(scenario, N=3):
  patient_symptomatics_dist = np.array([1 - scenario['p_sym'], scenario['p_sym']])
  patient_detection_dist = scenario['f_tsym']
  calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)

  len_simulation = 5000
  sim_results = []
  for x in range(len_simulation):
    if (x % 100 == 0):
      print(x)
    c0_symptomatic = get_samples(patient_symptomatics_dist)
    if (c0_symptomatic):
      c0_detection_date = get_samples(patient_detection_dist)
      c0_zipped = [(c0_detection_date, 0)]
    else:
      c0_zipped = []

    c1_infection_dist = normalize_lst([calc_Ai(i) for i in range(num_days)])
    c1_infection_date = get_samples(c1_infection_dist)
    c1_symptomatic = get_samples(patient_symptomatics_dist)
    if (c1_symptomatic):
      c1_detection_date = get_samples(patient_detection_dist)
      c1_zipped = [(c1_detection_date, 1)]
    else:
      c1_zipped = []

    i = c1_infection_date
    c2_infection_dist = normalize_lst([calc_Bij(i, j) for j in range(i, i + num_days)])
    c2_infection_dates = i + get_samples(c2_infection_dist, N)
    c2_symptomatics = get_samples(patient_symptomatics_dist, N)
    c2_infection_of_symptomatics_dates = c2_infection_dates[c2_symptomatics]
    c2_detection_dates = c2_infection_of_symptomatics_dates + get_samples(patient_detection_dist, len(c2_infection_of_symptomatics_dates))
    c2_zipped = list(zip(c2_detection_dates, 2 * np.ones(len(c2_detection_dates))))

    detection_dates_with_circles = c0_zipped + c1_zipped + c2_zipped
    sim_results.append(min(detection_dates_with_circles, key=lambda x: x[0]) - i)
    
  res_samples = get_samples(f_tres, len_simulation)
  return [(sim_results[i][0] + res_samples[i], sim_results[i][1]) for i in range(len(sim_results))]

In [0]:
scenario = dict(mode=1, f_tsym=f_tsym, f_tres=f_tres, p_sym=0.75, p_star=0, p_0=1.0, p_1=0.8, p_2=0.0)
stats = calculate_detection_statistics(scenario, 3)
detection_statistics = pd.DataFrame(stats, columns=['day', 'circle'])
circle0 = detection_statistics.day[detection_statistics.circle == 0]
circle1 = detection_statistics.day[detection_statistics.circle == 1]
circle2 = detection_statistics.day[detection_statistics.circle == 2]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [0]:
chance_of_descendants_to_quarantine_patient0 = lambda x, policy : len(detection_statistics.query('(0 <= circle <= {}) & (day <= {})'.format(policy, x)))/len(detection_statistics)

In [0]:
chance_of_descendants_to_quarantine_patient0(100, 0)

0.4816

In [0]:
policy = 1
filter_i = lambda i: (1 - len(df[(df.c > 0) & (df.c <= policy) & (df.a <= i)])/len(df))
filter_j = lambda i, j: filter_i(j - i) * (1 - len(df[(df.c == 1) & (df.a <= i)])/len(df))
filter_k = lambda i, j, k: filter_i(k - j) * (1 - len(df[(df.c == 1) & (df.a <= i)])/len(df))
calc_circles(scenario, filter_i, filter_j, filter_k)

(1, 0.9699782255890856, 0.6668273219832153, 0.4677329128663188)

In [0]:
calc_circles(scenario)

(1, 3.2850769219877054, 7.596000763723803, 17.932992281947055)

In [0]:
3.23/3.28, np.sqrt(7.4/7.59), np.cbrt(17.32/17.93)

(0.9847560975609757, 0.9874042039223774, 0.988528510277265)

In [0]:
px.bar(x=range(len(p_inf)), y=p_inf/p_inf.sum())

In [0]:
px.bar(x=list(range(len(f_tsym))), y=f_tsym)

In [0]:
def calculate_detection_statistics(scenario, N=3):
  patient_symptomatics_dist = np.array([1 - scenario['p_sym'], scenario['p_sym']])
  patient_detection_dist = scenario['f_tsym']
  calc_Ai, calc_Bij, calc_Cijk = get_calculators(scenario)

  len_simulation = 1000
  sim_results = []
  for x in range(len_simulation):
    if (x % 100 == 0):
      print(x)
    c0_symptomatic = get_samples(patient_symptomatics_dist)
    if (c0_symptomatic):
      c0_detection_date = get_samples(patient_detection_dist)
      c0_zipped = [(0, c0_detection_date, 0)]
    else:
      c0_zipped = [(0, None, 0)]

    c1_infection_dist = normalize_lst([calc_Ai(i) for i in range(num_days)])
    c1_infection_dates = get_samples(c1_infection_dist, N)
    c1_symptomatics = get_samples(patient_symptomatics_dist, N)
    c1_infection_of_symptomatics_dates = c1_infection_dates[c1_symptomatics == 1]
    c1_infection_of_asymptomatics_dates = c1_infection_dates[c1_symptomatics == 0]
    c1_zipped = []
    for elem in c1_infection_of_symptomatics_dates:
      c1_zipped.append((elem, elem + get_samples(patient_detection_dist), 1))
    for elem in c1_infection_of_asymptomatics_dates:
      c1_zipped.append((elem, None, 1))

    sim_results.append(c0_zipped + c1_zipped)
  return sim_results

In [0]:
histories = calculate_detection_statistics(scenario, N=3)

0
100
200
300
400
500
600
700
800
900


In [0]:
results = []
for history in histories:
  try:
    detecting_patient = min(filter(lambda x: x[1] != None, history), key=lambda x: x[1])
  except:
    results.append(None)
  infection_date_of_dp, detection_date_of_dp, circle_of_dp = detecting_patient
  detection_date_of_dp += get_samples(scenario['f_tres'])
  patient_in_1st_circle_already_infected = list(filter(lambda x: (x[2] == 1) and (x[0] <= detection_date_of_dp), history))
  patient_in_2nd_circle_already_infected = 0
  for patient in patient_in_1st_circle_already_infected:
    infection_date = patient[0]
    infection_dates_distribution = pd.Series(calc_Bij(infection_date, j) for j in range(0, infection_date + len(p_inf)))
    infection_dates_distribution /= infection_dates_distribution.sum()
    infection_dates_of_patient = get_samples(infection_dates_distribution, 3)
    patient_in_2nd_circle_already_infected += (infection_dates_of_patient <= detection_date_of_dp).sum()
  results.append((circle_of_dp, len(patient_in_1st_circle_already_infected), patient_in_2nd_circle_already_infected))
results = pd.DataFrame(results, columns=['circle','done_damage', 'done_damage2'])

In [0]:
results[results.circle == 1].describe()

,circle,done_damage,done_damage2
count,431.0,431.000000,431.000000
mean,1.0,2.879350,6.431555
std,0.0,0.378879,2.649147
min,1.0,1.000000,0.000000
25%,1.0,3.000000,5.000000
50%,1.0,3.000000,7.000000
75%,1.0,3.000000,9.000000
max,1.0,3.000000,9.000000


In [0]:
""